# Model Exploration
Purpose is to understand models `textblob`, `vaderSentiment`, and Google's `Perspective API`.<br>
By: Jonathan Lo<br>
Date: 1/16/23

In [1]:
# Model Imports
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Overhead Imports
from json import load
import requests as r

In [2]:
# Loading secrets
SECRETS = load(open('../secrets.json', 'r'))

## Model Setup

In [3]:
# Perspective API
def fetchPerspective(text: str):
    """ Sends a request to Perspective API for classification.
    """
    opts = {
        "q": text,
        "auth": f"Bearer {SECRETS['PerspectiveAPIKey']}"
    }
    res = r.get('<API URL>', params=opts)
    return res.json()

In [4]:
# TextBlob
def testTextBlobPolairty(text: str):
    """ Returns the polarity of the text using TextBlob.
    """
    return TextBlob(text).sentiment.polarity

def testTextBlobObjectivity(text: str):
    """ Returns the objectivity of the text using TextBlob.
    """
    return TextBlob(text).sentiment.subjectivity

In [5]:
# VaderSentiment
def testVaderSentiment(text: str):
    """ Returns the sentiment score given by VaderSentiment.
    """
    analyzer = SentimentIntensityAnalyzer()
    res = analyzer.polarity_scores(text)
    return res['compound']

## Sample Testing

In [6]:
sentences = [
    'You must be the change you wish to see in the world.',
    'Spread love everywhere you go. Let no one ever come to you without leaving happier.',
    'There is No War in Ba Sing Se.',
    'Money is the root of all evil.',
    'He bought the watch from the mall next to the Walmart.',
    'She brought her car into the car wash made for washing to wash her car.',
    'She sells sea shells by the sea shore.'
]
results = []

for sentence in sentences:
    result = {
        'tbPolairty': testTextBlobPolairty(sentence),
        'tbObjectivity': testTextBlobObjectivity(sentence),
        'vsScore': testVaderSentiment(sentence)
    }
    results.append(result)
    
results

[{'tbPolairty': 0.0, 'tbObjectivity': 0.0, 'vsScore': 0.4019},
 {'tbPolairty': 0.5, 'tbObjectivity': 0.6, 'vsScore': 0.0577},
 {'tbPolairty': 0.0, 'tbObjectivity': 0.0, 'vsScore': 0.4847},
 {'tbPolairty': -1.0, 'tbObjectivity': 1.0, 'vsScore': -0.6597},
 {'tbPolairty': 0.0, 'tbObjectivity': 0.0, 'vsScore': 0.0},
 {'tbPolairty': 0.0, 'tbObjectivity': 0.0, 'vsScore': 0.0},
 {'tbPolairty': 0.0, 'tbObjectivity': 0.0, 'vsScore': 0.0}]